In [1]:
# import data
logfile = open("../../Data/Logs/light1_first_se_log.txt","r")
loglines = logfile.readlines()
print (loglines)

['# Script started\n', 'SE0>rtc read\n', 'curr rtc time 1646297867.353394 s, 2022-03-03 08:57:47 \n', 'OK\n', 'SE0># Script delay (avoiding SAA)\n', 'SE0>script delayuntil 1646297935\n', 'OK\n', 'SE0># Enabling the PC and payload (and switching off with the timer of 30m for the payload and 3h for the PC)\n', 'SE0>csp txrx 4 14 600 000500302A\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0007000807\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0008000807\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0005010100\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp txrx 4 14 600 0007011400\n', 'Answer: 0000\n', 'Bytes: 2\n', 'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n', 'OK\n', 'SE0>csp t

In [20]:
for i in range(len(loglines)): # i is a number for every line in the log file
    k = loglines[i] 
    if k[0:5] in "SE0>#":
        print (k)

SE0># Script delay (avoiding SAA)

SE0># Enabling the PC and payload (and switching off with the timer of 30m for the payload and 3h for the PC)

SE0># Delay 2 minutes

SE0># Command ID 1 - Default Mode

SE0># Delay 2 seconds

SE0># Command ID 2 - Default Mode

SE0># Clear buffers

SE0># Command ID 9 - Noise Mode

SE0># Command ID 20 - Noise Mode

SE0># Command ID 3 - Operating Mode

SE0># Command ID 18 - Operating Mode

SE0># Command ID 1 - Default Mode

SE0># Command ID 2 - Default Mode

SE0># Switching the payloads OFF

SE0># Script finished

SE0>


In [15]:
for i in loglines:
    if i[0:5] in "SE0>#":
        print (i)
    

SE0># Script delay (avoiding SAA)

SE0># Enabling the PC and payload (and switching off with the timer of 30m for the payload and 3h for the PC)

SE0># Delay 2 minutes

SE0># Command ID 1 - Default Mode

SE0># Delay 2 seconds

SE0># Command ID 2 - Default Mode

SE0># Clear buffers

SE0># Command ID 9 - Noise Mode

SE0># Command ID 20 - Noise Mode

SE0># Command ID 3 - Operating Mode

SE0># Command ID 18 - Operating Mode

SE0># Command ID 1 - Default Mode

SE0># Command ID 2 - Default Mode

SE0># Switching the payloads OFF

SE0># Script finished

SE0>


In [5]:
with logfile as log:
    log2 = log.read()
    print (log2)

In [ ]:
# "Time,scenario"